# **Lab 4: Integration**
**Matteus Berg**

# **Abstract**

Three integration methods are implemented in this report:
* 2-point Gauss quadrature
* 3-point edge midpoint quadrature
* Monte Carlo quadrature

The integration methods are then verified by testing

#**About the code**

Written by Matteus Berg. Template distributed by Johan Hoffman

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them.

In [ ]:
# Load neccessary modules.
from google.colab import files

import math
import time
import numpy as np
import random

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

In this paper we implement three different quadrature methods. Quadratures are used to approximate integrals using a number of quadrature points. The quadratures that will be implemented are:
1. 2-point gauss quadrature over a unit interval
2. Three point edge midpoint quadrature over the reference triangle
3. Monte carlo quadrature over the unit interval

These three methods are implemented as functions. Tests are then formulated which call upon these functions. The tests are used to verify the correctness of the functions.



# **Method**

## 2-point Gauss quadrature

On page 242 in the book, the 2-point gauss quadrature for the interval $[-1, 1]$ is defined:

$\int_{-1}^1 p(x)dx = p(x_0)w_0 + p(x_1)w_1$

Where $w_0 = w_1 = 1$, $x_0 = 1/\sqrt{3}$, and $x_1 = -1/\sqrt{3}$.

However, we want for the unit interval. So the above formula will not work. We will have to find our own function $\phi(x)$ such that
1. $\int_0^1 \phi(x) dx = 0$
2. There exists q + 1 = 2 x-values in $[0, 1]$ where $\phi(x)=0$. These will be our two quadrature points

From page 241 in the book we then have the following derivation for a polynomial $p(x) \in P^{2q + 1}([0, 1])$. That is, a polynomial of degree 3:

$\int_0^1 p(x) = \int_0^1 \phi(x)s(x)dx + \int_0^1r(x) dx = \int_0^1 r(x)dx = \text{(from page 240 newton quotes quadratrature derivation)} = \int_0^1 \sum_{i=0}^q r(x_i)\lambda_i(x)dx = \sum_{i=0}^qr(x_i)\int_0^1\lambda_i(x)dx = \sum_{i=0}^q p(x_i)w_i$

Where $\lambda_i$ is the i:th lagrange polynomial defined on page 194 in the book. Additionally, $p(x) = \phi(x)s(x) + r(x)$

With all of this in mind, the polynomial $-2x^2+2x-\frac{1}{3}$ satisfies both criterions and will therefore be our $\phi(x)$ function. Solving for $\phi(x) = 0$ we get the following two solutions to use as our quadrature points: $x_0 = \frac{1}{2} - \frac{1}{\sqrt{12}}, x_1 = \frac{1}{2} + \frac{1}{\sqrt{12}}$.

To get the weights $w_0$ and $w_1$ we calculate $\int_0^1 \lambda_0(x) dx$ and $\int_0^1 \lambda_1(x) dx$ respectively. For $w_0$ we get that

$\int_0^1 \lambda_0(x) dx = \int_0^1 \frac{x-x_0}{x_0-x_1} dx = \int_0^1 \frac{x-\frac{\sqrt{12}+2}{2*\sqrt{12}}}{\frac{1}{2}+\frac{1}{\sqrt{12}}-\frac{1}{2} + \frac{1}{\sqrt{12}}} dx = \frac{1}{2}$

We get that $w_1$ also is equal to $\frac{1}{2}$.

In summary, the final 2-point gauss quadrature is $\int_0^1p(x)dx = p(x_0)w_0 + p(x_1)w_1$

Where $w_0 = w_1 = \frac{1}{2}, x_0 = \frac{1}{2} - \frac{1}{\sqrt{12}}, x_1 = \frac{1}{2} + \frac{1}{\sqrt{12}}$

## 3-point edge midpoint quadrature over a reference triangle
For this function we get from the book on page 243 that the weights $w_0, w_1$ and $w_2$ are all equal to $1/6$. Furthermore, the book describes the quadrature points being the "midpoints of the three edges of the reference triangle". Meaning we get that $x_0=(0,\frac{1}{2}), x_1 =(\frac{1}{2}, 0), x_2=(\frac{1}{2}, \frac{1}{2})$.

As usual we have $q+1$ quadrature points, meaning that $q=2$. Since we are now integrating in both x and y, we get a double integral. Integrating over the reference triangle can be described with $\int_0^1 \int_0^{1-y} p(x) dx dy$. So

$\int_0^1 \int_0^{1-y} p(\textbf{x}) dx dy = \sum_{i=0}^2 p(\textbf{x}_i)w_i = p(\textbf{x}_0)w_0 + p(\textbf{x}_1)w_1 + p(\textbf{x}_2)w_2$

## Monte Carlo quadrature over a unit interval
Monte carlo integration is defined on page 268 in the book:

$\int_D g(x)dx \approx \frac{1}{n} \sum_{i=1}^n g(x_i)|D|$

Where $D$ is the domain, $n$ the amount of randomly sampled quadrature points and $x_i$ a sample point. This quadrature method uses randomisation to solve deterministic integrals.

In [ ]:

# 2 point gauss quadrature
def twoPointGaussUnitInterval(p):
  w0 = 1/2
  w1 = 1/2
  x0 = 1/2 - 1/math.sqrt(12)
  x1 = 1/2 + 1/math.sqrt(12)
  return p(x0)*w0 + p(x1)*w1

def threeMidpointReferenceTriangle(p):
  w0 = 1/6
  w1 = 1/6
  w2 = 1/6
  x0 = np.array([0, 1/2])
  x1 = np.array([1/2, 0])
  x2 = np.array([1/2, 1/2])

  return p(x0)*w0 + p(x1)*w1 + p(x2)*w2

def monteCarloUnitInterval(p, n):
  a = 0
  b = 1
  D = b - a
  x = np.zeros(n)
  random.seed(28)
  sum = 0

  for i in range(0, n):
    x[i] = random.uniform(a, b)
    sum = sum + p(x[i]) * D
  return sum/n


# test 2-point gauss quadrature over unit interval
poly = lambda x : x*x*x + x*x
print("2-point Gauss quadrature test for function p(x) = x^3 + x^2")
print("Expected result: ", 7/12)
print("Actual result: ", twoPointGaussUnitInterval(poly))
print(" ")
# test 3-point midpoint quadrature over reference triangle
poly = lambda x : x[0]*x[0] + x[1]*x[1] + x[0]*x[1] + x[0] + x[1]
print("3-point midpoint quadrature test for function p(xhat) = x^2 + y^2 + xy + x + y")
print("Expected result: ", 13/24)
print("Actual result: ", threeMidpointReferenceTriangle(poly))
print(" ")

# test monte carlo quadrature over unit interval
poly = lambda x : x*x*x
print("monte carlo quadrature convergence test for function p(x) = x^3")
print("Expected result: ", 1/4)
print("Result for 1/sqrt(n) = 0. <=> n=2        ", monteCarloUnitInterval(poly, 2))
print("Result for 1/sqrt(n) = 0.0 <=> n=100:    ", monteCarloUnitInterval(poly, 100))
print("Result for 1/sqrt(n) = 0.00 <=> n=10 000:", monteCarloUnitInterval(poly, 10000))


2-point Gauss quadrature test for function p(x) = x^3 + x^2
Expected result:  0.5833333333333334
Actual result:  0.5833333333333333
 
3-point midpoint quadrature test for function p(xhat) = x^2 + y^2 + xy + x + y
Expected result:  0.5416666666666666
Actual result:  0.5416666666666666
 
monte carlo quadrature convergence test for function p(x) = x^3
Expected result:  0.25
Result for 1/sqrt(n) = 0. <=> n=2         0.0018408728170225975
Result for 1/sqrt(n) = 0.0 <=> n=100:     0.2064294825726514
Result for 1/sqrt(n) = 0.00 <=> n=10 000: 0.2502733386457965


# **Results**

From running the code in the method. We can se that
1. 2-point gauss quadrature precisely approximates a 3rd degree polynomial p(x) in the unit interval
2. 3-point midpoint quadrature precisely approximates a 2nd degree polynomial p(x,y)
3. Monte carlo quadrature converges to the exact integral value with the rate $\frac{1}{\sqrt{n}}$. Where $n$ is the amount of sampled quadrature points

# **Discussion**


All implemented algorithms behaved as expected. Running the monte carlo integral for n = 10^8 to further verify the convergence rate was attempted. However, the execution time became longer than 10 seconds then, so that test was exempted from the code.